In [10]:
import json
with open('countries.json') as f:
    countries = json.load(f)

In [12]:
del countries["EG"]
del countries["SA"]

In [4]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
import re
from selenium import webdriver
import urllib.request
import urllib.error
from selenium.webdriver.support import expected_conditions as EC
import time
from selenium import webdriver
import time
from selenium import webdriver
from datetime import datetime
import urllib.request
from datetime import datetime
import sqlite3
import requests



In [5]:
 webdriver.Chrome(service=Service(ChromeDriverManager().install()))

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 105.0.5195
[WDM] - Get LATEST chromedriver version for 105.0.5195 google-chrome
[WDM] - About to download new driver from https://chromedriver.storage.googleapis.com/105.0.5195.52/chromedriver_win32.zip
[WDM] - Driver has been saved in cache [C:\Users\asd_v\.wdm\drivers\chromedriver\win32\105.0.5195.52]


<selenium.webdriver.chrome.webdriver.WebDriver (session="613332e9a20c546dc857b111cd460dbb")>

In [36]:
# driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
def save_log(log):
    print(log)
    with open('log.txt', 'a+') as my_data_file:
        my_data_file.write(f'{log}\n')
#log in facebook

def open_link(search_term,country= "EG",start_date = None,end_date=None,media_type='video'):
    link = f'https://www.facebook.com/ads/library/?active_status=active&ad_type=all&country={country}&q={search_term}&sort_data[direction]=desc&sort_data[mode]=relevancy_monthly_grouped&start_date[min]={start_date}&start_date[max]={end_date}&search_type=keyword_unordered&media_type=all&media_type={media_type}'
    driver.get(link)
    time.sleep(2)
    
def open_page(page_id,country= "EG",start_date = None,end_date=None):
    link= f'https://www.facebook.com/ads/library/?active_status=active&ad_type=all&country={country}&view_all_page_id={page_id}&sort_data[direction]=desc&sort_data[mode]=relevancy_monthly_grouped&search_type=page&media_type=video'
    driver.get(link)
    time.sleep(2)
    
def get_ads_number():
    try:
        text = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR,'div[class="_7lca"]'))).text
        ADS_count = re.findall('(\d+) result', text)[0]
        tries = 0
        while ADS_count == '0' and tries < 2:
            tries+=1
            driver.refresh()
            time.sleep(5)
            text = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR,'div[class="_7lca"]'))).text
            ADS_count = re.findall('(\d+) result', text)[0]
            if ADS_count != '0':
                save_log(f"should not be 0,{ADS_count}")
                break
            save_log(f"should be 0, {ADS_count},{tries}")
            time.sleep(300)
    except:
        ADS_count = 0
    return int(ADS_count)

def scroll_down(limit = 'no limit'):
    if limit == 'no limit':
         #scroll to end 
        # Get scroll height
        last_height = driver.execute_script("return document.body.scrollHeight")

        while True:
            # Scroll down to bottom
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # Wait to load page
            time.sleep(2)

            # Calculate new scroll height and compare with last scroll height
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height
    
    #scroll enough 
    else:
        ads = driver.find_elements(By.CSS_SELECTOR, value = 'div[class="_99s5"]')
        temp_ads = 0
        while (len(ads) < limit and temp_ads < len(ads)):
            temp_ads = len(ads)
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(3)
            ads = driver.find_elements(By.CSS_SELECTOR, value = 'div[class="_99s5"]')
        time.sleep(2)
        
def find_ID(element):
    try:
        return WebDriverWait(element,10).until(EC.presence_of_element_located((By.XPATH, './div/div[1]/div/div[1]/div[4]/div/div/span'))).text[4:]
    except:
        return WebDriverWait(element,10).until(EC.presence_of_element_located((By.XPATH, './div/div[1]/div/div[1]/div[5]/div/div/span'))).text[4:]


def find_start_date(element):
    text = WebDriverWait(element, 10).until(EC.presence_of_element_located((By.XPATH,'./div/div[1]/div/div[1]/div[2]/span'))).text#[19:]
    date = re.findall('\d+ \S+ \d+',text)[0]
    return datetime.strptime(date,"%d %b %Y" ).strftime("%Y-%m-%d")


def find_profile_pic(element):
    try:
        link = WebDriverWait(element, 10).until(EC.presence_of_element_located((By.XPATH,'.//img[@class="_8nqq img"]'))).get_attribute('src')
        name = re.findall('\d+_\d+_\d+_n',link)[0] + '.png'
        #download
#         img_data = requests.get(link).content
#         with open(name, 'wb') as handler:
#             handler.write(img_data)
        # get path
        full_path = '' + name

        return link
    except:
        return  ""  #'84702798_579370612644419_4516628711310622720_n.png'


def find_links(element):
    links = "\n".join([a.get_attribute('href') for a in element.find_elements(By.CSS_SELECTOR,'a')[1:] ])
    links = urllib.parse.unquote(links).replace('https://l.facebook.com/l.php?u=', '')
    links = re.findall('(https.+)&',links)
    return "\n".join(links)

def find_ad_videos(element):
    try:
        vids = WebDriverWait(element,10).until(EC.presence_of_all_elements_located((By.XPATH,'.//video')))
        vids_links = [a.get_attribute('src') for a in vids ]
        names = []
        for video in vids_links:
            name = re.findall('\d+_\d+_\d+_n',video)[0] + '.mp4'
#             urllib.request.urlretrieve(video, name)
            names.append(name)
#         "\n".join(names)
        return "\n".join(vids_links)
    except:
        return "No Videos found"

def find_content(element):
    return WebDriverWait(element,10).until(EC.presence_of_element_located((By.XPATH,'./div/div[3]/div/div/div[2]/div'))).text


def find_footer(element):
    try:
        footer = WebDriverWait(element, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR,'div[class="_8jgz _8jg_"]')))
        action = WebDriverWait(footer, 10).until(EC.presence_of_element_located((By.XPATH,'./div[2]/div/div/span/div/div/div'))).text
        return footer.text , action
    except:
        return "No Footer" , "No Action"


def find_page_name(element):
    return WebDriverWait(element,10).until(EC.presence_of_element_located((By.XPATH, './div/div[3]/div/div/div[1]/div/div/div'))).text.replace("\nSponsored", "")

def find_ads_occurence(element):
    try:
        return int(element.find_element(By.XPATH, value ='./div/div[1]/div/div[1]/div[5]/span/strong').text.split(' ')[0])
    except:
        return '1'

def click_see_ad_details(element):
    click = WebDriverWait(element, 20).until(EC.element_to_be_clickable((By.XPATH,'./div/div[2]/div/span/div/div/div')))
    driver.execute_script("arguments[0].click();", click)
    time.sleep(0.5)
    return WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR,'div[class="_5aat _4-hy uiLayer _3qw"]')))

def get_page_data(element):
    #from the about only 
    
    #MAIN PAGE ELEMENT
    main_element = click_see_ad_details(element)

    #ABOUT THE PAGE ELEMENT
    about_the_page = WebDriverWait(main_element, 10).until(EC.presence_of_element_located((By.XPATH,'./div[2]/div/div/div/div/div[3]/span/div[2]/div/div')))  

    FB_ID = "NO Facebook ID found"
    Insta_ID = "NO insta ID found"
    page_likes = 0
    insta_followers = 0

    try:
        #all pages
        pages = WebDriverWait(about_the_page, 2).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR,'div[class="hael596l alzwoclg c61n2bf6 q46jt4gp bq6c9xl4 r9sb4e79"]')))
        for s in pages:    
            text = s.find_element(By.XPATH,'.').text.replace(',','')
            #facebook 
            if re.search('like', text):
                FB_ID = s.find_element(By.XPATH,'./div[2]/div[1]').text
                page_likes = int(re.findall('(\d+) like' , text)[0])
            if re.search('follower', text):
                Insta_ID = s.find_element(By.XPATH,'./div[2]/div[1]').text
                insta_followers = int(re.findall('(\d+) follower' , text)[0])
    except:
        pass
    
    
    #static ID 
    static_ID = "NO static ID found"
    #open the page 
    # pages_IDS.append(static_ID)
    # save_log(f'page IDS {len(pages_IDS)}')
    try:
        page_link = WebDriverWait(main_element, 10).until(EC.presence_of_element_located((By.XPATH,'./div[2]/div/div/div/div/div[3]/span/div[2]/div/div/div[1]/div/a'))).get_attribute('href')
        static_ID = re.findall(('view_all_page_id=(\d+)'),page_link)[0]
    #     # Open a new window
    #     driver.execute_script("window.open('');")

    #     # Switch to the new window and open new URL
    #     driver.switch_to.window(driver.window_handles[1])
    #     open_page(static_ID)
    #     # driver.get(page_link)
    #     ADS_count = get_ads_number()
    #     driver.close()

    #     # Switching to old tab
    #     driver.switch_to.window(driver.window_handles[0])
    except:
        pass
    #     ADS_count = 0
    time.sleep(0.5)
    webdriver.ActionChains(driver).send_keys(Keys.ESCAPE).perform()
    time.sleep(1) 
#     print(FB_ID, page_likes,Insta_ID,insta_followers, static_ID)
    return FB_ID, page_likes,Insta_ID,insta_followers,static_ID




def start_save(search_term,country= "EG",start_date = None,end_date=None,media_type='video',limit = 'no limit'):
    open_link(search_term,country,start_date,end_date,media_type)
    scroll_down(limit)
    count = 0
    # elements = []
    elements = driver.find_elements(By.CSS_SELECTOR, value = 'div[class="_99s5"]')
    while len(elements)==0:
        save_log("no elements")
        time.sleep(300)
        driver.refresh()
        elements = driver.find_elements(By.CSS_SELECTOR, value = 'div[class="_99s5"]')
    page_IDS = []
    if limit == "no limit":
        limit = len(elements)+1
    for element in elements[0:limit]:
        count =+ 1
        time_now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        days = datetime.now().strftime("%d/%m/%Y")
        AD_ID = find_ID(element)
        save_log(AD_ID)
        Started_date = find_start_date(element)
        profile_pic = find_profile_pic(element)
        links = find_links(element)
        videos = find_ad_videos(element)
        content = find_content(element)
        Footer_text , Footer_action = find_footer(element)
        Page_name = find_page_name(element)
        AD_occurance = find_ads_occurence(element)
#         FB_ID, page_likes,Insta_ID,insta_followers, static_ID,ADS_count
        Facebook_ID, Page_likes,instgram_ID , insta_followers, static_ID  = get_page_data(element)
        page_IDS.append(static_ID)
        save_log(time_now)
        # if Ads_count == 0 or Ads_count == "0":
        #     time.sleep(300)
            # driver.refresh()
            # Facebook_ID, Page_likes,instgram_ID , insta_followers, static_ID, Ads_count  = get_page_data(element)
        #save to database   
        try:
        #insert if not exist 
            conn = sqlite3.connect('FaceBoookADS.db')
            c = conn.cursor()
            c.execute('''INSERT INTO ads VALUES 
            (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?);'''
                ,(AD_ID ,
                Started_date ,
                profile_pic ,
                links ,
                videos ,
                content ,
                Footer_text ,
                Footer_action ,
                Page_name ,
                AD_occurance ,
                Facebook_ID ,
                Page_likes ,
                instgram_ID ,
                insta_followers ,
                static_ID ,
                0 ,
                0 ,
                days ,
                time_now ,
                1 ,
                search_term ,
                ))
            conn.commit()
            conn.close()
        except:
            #update the data 
            conn = sqlite3.connect('FaceBoookADS.db')
            c = conn.cursor()
            c.execute('''UPDATE ads SET 
            AD_occurance =?,
            Page_likes =?,
            insta_followers =?,
            Ads_count =?,
            cumulative_ads_count = cumulative_ads_count + ?,
            date = ? || ',' || date ,
            days = ?  || ',' || days ,
            hits = hits + 1 , 
            search_term = search_term || ',' || ?   
            where AD_ID = ?'''
              ,(AD_occurance ,
                Page_likes ,
                insta_followers ,
                0 ,
                0,
                time_now,
                days,
                search_term,
               AD_ID))
            conn.commit()
            conn.close()
    return page_IDS    


# terms = '''عطور
# perfume
# عطور
# '''.split()

# for term in terms:
#     driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
#     driver.get('https://www.facebook.com')
#     WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR,'input[name="email"]'))).send_keys('drazahmed1969@gmail.com')
#     WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR,'input[name="pass"]'))).send_keys('lordASD4facebook@@' , Keys.ENTER)
#     time.sleep(0.5)
#     save_log(f"started the term: {term}")
#     page_IDS = start_save(term, limit=60)
#     for page_ID in list(set(page_IDS)):
#         open_page(page_ID)
#         Ads_count = get_ads_number()
#         conn = sqlite3.connect('FaceBoookADS.db')
#         c = conn.cursor()
#         c.execute('''UPDATE ads SET 
#         Ads_count =?,
#         cumulative_ads_count = cumulative_ads_count + ?
#         where static_id = ?'''
#         ,(
#             Ads_count ,
#             Ads_count,
#             page_ID))
#         conn.commit()
#         conn.close()
#     time.sleep(0.5)
#     driver.close()
    

# sqliteConnection = sqlite3.connect('FaceBoookADS.db')
# cursor = sqliteConnection.cursor()
# sqlite_select_query = f"""SELECT static_ID FROM ads where Ads_count = 0 """
# cursor.execute(sqlite_select_query)
# records = cursor.fetchall()
# page_IDS = [i[0] for i in list(set(records))]
# cursor.close()
# for page_ID in page_IDS:
#     open_page(page_ID)
#     Ads_count = get_ads_number()
#     conn = sqlite3.connect('FaceBoookADS.db')
#     c = conn.cursor()
#     c.execute('''UPDATE ads SET 
#     Ads_count =?,
#     cumulative_ads_count = cumulative_ads_count + ?
#     where static_id = ?'''
#     ,(
#     Ads_count ,
#     Ads_count,
#     page_ID))
#     conn.commit()
#     conn.close()

In [42]:
options = webdriver.ChromeOptions()
options.add_argument("--window-size=1920,1080")
options.add_argument("--headless")
options.add_argument("--disable-gpu")
options.add_argument(
    "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36")
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),options=options )
# driver.get('https://www.facebook.com')
# WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR,'input[name="email"]'))).send_keys('drazahmed1969@gmail.com')
# WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR,'input[name="pass"]'))).send_keys('lordASD4facebook@@' , Keys.ENTER)
time.sleep(0.5)
open_link('عطور')
elements = driver.find_elements(By.CSS_SELECTOR, value = 'div[class="_99s5"]')
print(len(elements))

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 105.0.5195
[WDM] - Get LATEST chromedriver version for 105.0.5195 google-chrome
[WDM] - Driver [C:\Users\asd_v\.wdm\drivers\chromedriver\win32\105.0.5195.52\chromedriver.exe] found in cache


24


In [2]:
# Create database
import sqlite3
conn = sqlite3.connect('FaceBoookADS.db')
c = conn.cursor()
try:
    c.execute('''DROP TABLE ads''')
except:
    pass
c.execute('''
CREATE TABLE ads
(AD_ID TEXT PRIMARY KEY, 
sss TEXT)''')
conn.commit()
conn.close()

In [3]:
conn = sqlite3.connect('FaceBoookADS.db')
c = conn.cursor()
c.execute('''INSERT INTO ads VALUES 
(?, ?)'''
          , ('id1', 'text1'))
conn.commit()
conn.close()

TEST area


In [6]:
# #log in 
# driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
# driver.get('https://www.facebook.com')
# WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR,'input[name="email"]'))).send_keys('drazahmed1969@gmail.com')
# WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR,'input[name="pass"]'))).send_keys('lordASD4facebook@@' , Keys.ENTER)
# time.sleep(0.5)

In [30]:
def find_ID(element):
    try:
        return WebDriverWait(element,10).until(EC.presence_of_element_located((By.XPATH, './div/div[1]/div/div[1]/div[4]/div/div/span'))).text[4:]
    except:
        return WebDriverWait(element,10).until(EC.presence_of_element_located((By.XPATH, './div/div[1]/div/div[1]/div[5]/div/div/span'))).text[4:]

In [25]:
def find_links(element):
    links = "\n".join([a.get_attribute('href') for a in element.find_elements(By.CSS_SELECTOR,'a')[1:] ])
    links = urllib.parse.unquote(links).replace('https://l.facebook.com/l.php?u=', '')
    links = re.findall('(https.+)&',links)
    return 

In [26]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))


[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 105.0.5195
[WDM] - Get LATEST chromedriver version for 105.0.5195 google-chrome
[WDM] - Driver [C:\Users\asd_v\.wdm\drivers\chromedriver\win32\105.0.5195.52\chromedriver.exe] found in cache


In [59]:
def find_footer(element):
    try:
        footer = WebDriverWait(element, 2).until(EC.presence_of_element_located((By.XPATH,'./div/div[3]/div/div/div[2]/div[3]')))
        action = WebDriverWait(footer, 2).until(EC.presence_of_element_located((By.XPATH,'./div[2]/div/div/span/div/div/div'))).text
        links = "\n".join([a.get_attribute('href') for a in element.find_elements(By.CSS_SELECTOR,'a')])
#         links = "\n".join([a.get_attribute('href') for a in element.find_elements(By.XPATH,'//a[@href]')])
        print(links)
        links = urllib.parse.unquote(links).replace('https://l.facebook.com/l.php?u=', '')
        links = re.findall('(https.+)&',links)
        return footer.text , action , "\n".join(links)
    except:
        return "No Footer" , "No Action", ""

In [60]:
elements = driver.find_elements(By.CSS_SELECTOR, value = 'div[class="_99s5"]')
for element in elements:
    print(find_ID(element))
    print(find_footer(element))

    

1238577373621555


TimeoutException: Message: 
Stacktrace:
Backtrace:
	Ordinal0 [0x007CDF13+2219795]
	Ordinal0 [0x00762841+1779777]
	Ordinal0 [0x0067423D+803389]
	Ordinal0 [0x006A3025+995365]
	Ordinal0 [0x006A31EB+995819]
	Ordinal0 [0x00699531+955697]
	Ordinal0 [0x006BE844+1108036]
	Ordinal0 [0x006994B4+955572]
	Ordinal0 [0x006BEA14+1108500]
	Ordinal0 [0x006CF192+1175954]
	Ordinal0 [0x006BE616+1107478]
	Ordinal0 [0x00697F89+950153]
	Ordinal0 [0x00698F56+954198]
	GetHandleVerifier [0x00AC2CB2+3040210]
	GetHandleVerifier [0x00AB2BB4+2974420]
	GetHandleVerifier [0x00866A0A+565546]
	GetHandleVerifier [0x00865680+560544]
	Ordinal0 [0x00769A5C+1808988]
	Ordinal0 [0x0076E3A8+1827752]
	Ordinal0 [0x0076E495+1827989]
	Ordinal0 [0x007780A4+1867940]
	BaseThreadInitThunk [0x7653FA29+25]
	RtlGetAppContainerNamedObjectPath [0x77957B5E+286]
	RtlGetAppContainerNamedObjectPath [0x77957B2E+238]


In [11]:
list1 = []
"\n".join(list1)

''

In [20]:
import sqlite3
sqliteConnection = sqlite3.connect('FaceBoookADS2.db')
cursor = sqliteConnection.cursor()
sqlite_select_query = f"""SELECT static_ID FROM ads where Ads_count = 0 """
cursor.execute(sqlite_select_query)
records = cursor.fetchall()
page_IDS = [i[0] for i in list(set(records))]
cursor.close()
for i in list(set(records)):
    open_page(page_ID)
    Ads_count = get_ads_number()
    page_ID = i[0]
    conn = sqlite3.connect('FaceBoookADS2.db')
    c = conn.cursor()
    c.execute('''UPDATE ads SET 
    Ads_count =?,
    cumulative_ads_count = cumulative_ads_count + ?
    where static_id = ?'''
    ,(
    Ads_count ,
    Ads_count,
    page_ID))
    conn.commit()
    conn.close()

In [90]:
import sqlite3
sqliteConnection = sqlite3.connect('FaceBoookADS.db')
cursor = sqliteConnection.cursor()
col = 'AD_ID'
sqlite_select_query = f"""SELECT links FROM ads where links is not '' """

cursor.execute(sqlite_select_query)
records = cursor.fetchall()
print(len(records))
print(records[0:20])
cursor.close()
# for i in list(set(records)):
#     open_page(page_ID)
#     Ads_count = get_ads_number()
#     page_ID = i[0]
#     conn = sqlite3.connect('FaceBoookADS2.db')
#     c = conn.cursor()
#     c.execute('''UPDATE ads SET 
#     Ads_count =?,
#     cumulative_ads_count = cumulative_ads_count + ?
#     where static_id = ?'''
#     ,(
#     Ads_count ,
#     Ads_count,
#     page_ID))
#     conn.commit()
#     conn.close()

626
[('https://www.esmiskin.com/?fbclid=IwAR3OIBrxYA8TdNqvgUJFopUKTsx3OCfokuptzsud_xZQJ0FLVS79gPlFe84',), ('https://www.esmiskin.com/?fbclid=IwAR02GjHUuGZW0XcMb-m8a34fi1AZCcYjZOk0-6uYCv2A65FknbX2tkidEUk',), ('https://www.ibleromptly.com/products/2022-new-year-promotion---buy-2-get-1-free-poreless-deep-cleanse-green-tea-mask?fbclid=IwAR00H2UfytrZ4F0UxdfN-QqKVFKRRIIMAGqVfNmDEow8VMD9CC39pgQi508',), ('https://www.ibleromptly.com/products/2022-new-year-promotion---buy-2-get-1-free-poreless-deep-cleanse-green-tea-mask-2?fbclid=IwAR02GjHUuGZW0XcMb-m8a34fi1AZCcYjZOk0-6uYCv2A65FknbX2tkidEUk',), ('https://glamtool.co/products/plasma-pen?fbclid=IwAR3PG3tZrc1rN7vsM28ESG6VB6PC3XC8NfDmRFNpOX_p4y-1CmFodMkjLpY\nhttps://glamtool.co/?fbclid=IwAR0dLoMedWEuhGcux5WuyBJDa1agVxS7w_IDQFX5PZbGpQCdC5KwJ4kjgy4',), ('https://glamtool.co/products/plasma-pen?fbclid=IwAR3WhLXOtTesmJ8UnuW5B2k-HJZkOX4CnfQPGxjzm48-5yonhyFxIJWS52Q\nhttps://glamtool.co/?fbclid=IwAR26RY-FFoiPw3u6q5WN41ur9MtpZC0AW9hAjx4f01_8NEs1DHHmtccLeyk

In [9]:
from urllib.parse import urlparse
from collections import Counter
import itertools
import sqlite3
from itertools import compress

In [65]:
from urllib.parse import urlparse
from collections import Counter
import itertools
import sqlite3
from itertools import compress
sqliteConnection = sqlite3.connect('FaceBoookADS.db')
cursor = sqliteConnection.cursor()
sqlite_select_query = f"""SELECT days FROM ads where days is not '' """
cursor.execute(sqlite_select_query)
records = cursor.fetchall()
cursor.close()
days = []
for rec in records:
    days.append(len(set(rec[0].split(','))))
    
days_running = 2
exec(f"cond_list = [True if i > {days_running}else False for i in days]")
ads = compress(records,cond_list)
# domains = [ urlparse(i[0]).netloc for i in records]
# advertisers = [i[0] for i in records]
# sorted_dict = dict(sorted(Counter(advertisers).items(), key=lambda item: item[1],reverse=True))
# dict(itertools.islice(sorted_dict.items(), 5))

### pics save and name

In [113]:
import re
import requests

In [109]:
urls = ['https://scontent.fcai19-1.fna.fbcdn.net/v/t39.35426-6/307748491_762222315067647_8404398243781207614_n.jpg?stp=dst-jpg_s60x60&_nc_cat=103&ccb=1-7&_nc_sid=cf96c8&_nc_ohc=tmTPQ9CQdWkAX8AXhQC&_nc_ht=scontent.fcai19-1.fna&oh=00_AT-LsLHgR7UbDDhLVTRw8WVkEWkaCtX6w_L-clCQXpVzDw&oe=632CB999','https://scontent.fcai19-1.fna.fbcdn.net/v/t39.35426-6/306787864_3326779997606937_3989988158988851504_n.jpg?stp=dst-jpg_s60x60&_nc_cat=107&ccb=1-7&_nc_sid=cf96c8&_nc_ohc=rS5NvDi2ntcAX_u_HbQ&_nc_ht=scontent.fcai19-1.fna&oh=00_AT8O2iY46ND7haIXRYai8M53jeC5IeBB39nl4zZxCcB4iQ&oe=632BBD09']

In [115]:
for link in urls:
    name = re.findall('\d+_\d+_\d+_n',link)[0] + '.png'
    img_data = requests.get(link).content
    with open(f'./media/pics/{name}', 'wb') as handler:
        handler.write(img_data)

In [3]:
import cv2
import datetime
import re
import urllib
vids_links = ['https://video.fcai19-1.fna.fbcdn.net/v/t42.1790-2/308535829_625066009068427_7533256486936282957_n.?_nc_cat=104&ccb=1-7&_nc_sid=cf96c8&_nc_ohc=OX1-4OahkSgAX9ZVC4p&_nc_ht=video.fcai19-1.fna&oh=00_AT_Q-mtoI7jEQLhFabtQuAUSsO-WAgo66H_n8R_tdw4oew&oe=632DD8B4']
for video in vids_links:
    name = re.findall('\d+_\d+_\d+_n',video)[0] + '.mp4'
    data = cv2.VideoCapture(video)
    frames = data.get(cv2.CAP_PROP_FRAME_COUNT)
    fps = data.get(cv2.CAP_PROP_FPS)
    seconds = round(frames / fps)
    video_time = datetime.timedelta(seconds=seconds)
    print(f"video time: {video_time}")
    if seconds < 40:
        urllib.request.urlretrieve(video, f'./media/vids/{name}')


video time: 0:00:37


#### django code

In [48]:
import os, sys
PWD = os.path.abspath(os.getcwd())
from collections import Counter
import itertools
from itertools import compress
from urllib.parse import urlparse
from collections import Counter
from ad.models import facebook_pages
from ad.models import Ads
from ad.models import search_term
from functools import reduce
import operator
from django.db.models import Q
os.chdir(PWD)
sys.path.insert(0, os.getenv('PWD'))
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "local_settings.py")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
import django
django.setup()

In [ ]:
ADS = ADS.filter(reduce(operator.or_, (Q(page_name__icontains=x) for x in req.getlist('Advertiser')))).filter(favourite=True).order_by(sort_by)

In [59]:
pages = facebook_pages.objects.filter(favorites=True).values_list('static_ID')
pages = [page[0] for page in pages]
Ads.objects.filter( Q(static_id__in=pages) | Q(favourite = True))

<QuerySet [<Ads: Ads object (1211099893065888)>, <Ads: Ads object (3128904247420992)>, <Ads: Ads object (443525747593840)>, <Ads: Ads object (453111666780817)>, <Ads: Ads object (772180670670202)>, <Ads: Ads object (623931189380931)>, <Ads: Ads object (783433472895048)>, <Ads: Ads object (655823679113243)>, <Ads: Ads object (841204193575386)>, <Ads: Ads object (1580487522354514)>, <Ads: Ads object (765753994652719)>, <Ads: Ads object (449003133858585)>, <Ads: Ads object (625598062340476)>, <Ads: Ads object (811861700001091)>, <Ads: Ads object (2263301667200912)>, <Ads: Ads object (538114064748212)>, <Ads: Ads object (806163757087685)>, <Ads: Ads object (827115091645294)>, <Ads: Ads object (3200198710240359)>, <Ads: Ads object (2906726266137999)>, '...(remaining elements truncated)...']>

In [26]:
pages = facebook_pages.objects.filter(favorites=True)
for page in pages:
    try:
        search_term(search_term=page_name, search_type='Page',active=True,static_ID =static_id ).save()
    except:
        pass

In [2]:
state = facebook_pages.objects.get(static_ID=static_id).favorites
facebook_pages.objects.filter(static_ID=static_id).update(favorites = not(state))
page_name = facebook_pages.objects.get(static_ID=static_id).page_name
if state  == False:
    try:
        follow_page = search_term(search_term=page_name, search_type='Page',active=True,static_ID =static_id )
        follow_page.save()
    except:
        pass
elif state == True:
    try:
        search_term.objects.get(static_ID=static_id).delete()
    except:
        pass

In [5]:
ADS = Ads.objects.filter(favourite=True).order_by('-ads_count')

In [6]:
domains_side = [link.links for link in ADS]#ADS.values_list('links')
domains_side = [ urlparse(i).netloc for i in domains_side]
domains_side = dict(sorted(Counter(domains_side).items(), key=lambda item: item[1],reverse=True))
domains_side.pop('', 'key not found')
domains_side

{'uk.drsquatch.com': 1, 'www.esmiskin.com': 1}

In [9]:
Ads.objects.all().update(favourite=False)

1340

### save  all facebook pages

In [8]:
import sqlite3
sqliteConnection = sqlite3.connect('FaceBoookADS.db')
cursor = sqliteConnection.cursor()
col = 'AD_ID'
sqlite_select_query = f"""SELECT static_id ,page_name FROM ads """
sqlite_select_query
cursor.execute(sqlite_select_query)
records = cursor.fetchall()
# records = [i[0] for i in records]
print(len(records))
cursor.close()

sqliteConnection = sqlite3.connect('FaceBoookADS.db')
cursor = sqliteConnection.cursor()
for record in records:
    try:
        cursor.execute('''INSERT INTO facebook_pages VALUES 
        (?, ?,?)'''
                  , (record[0],record[1],False))
        sqliteConnection.commit()
    except:
        pass
cursor.close()

1343


In [12]:
cursor.close()

In [65]:
facebook_pages.objects.get(pk='118075261668462').favorites == True

True

In [52]:
facebook_pages.objects.get(pk=int(locality['state']))filter(static_ID = ).values_list('favorites')[0][0]

<facebook_pages: facebook_pages object (100327312272304)>

In [14]:
# conn.close()
cursor.close()
sqliteConnection.close()

('798240993621809', 'esmi Skin Minerals', False)
('798240993621809', 'esmi Skin Minerals', False)
('1149310848422401', 'One Clinic Laser & Skin Care', False)
('1149310848422401', 'One Clinic Laser & Skin Care', False)
('111235721585655', 'Ibleromptly', False)
('111235721585655', 'Ibleromptly', False)
('111235721585655', 'Ibleromptly', False)
('111235721585655', 'Ibleromptly', False)
('101424012697046', 'GlamTool', False)
('101424012697046', 'GlamTool', False)
('101424012697046', 'GlamTool', False)
('101424012697046', 'GlamTool', False)
('101424012697046', 'GlamTool', False)
('101424012697046', 'GlamTool', False)
('110615337321740', 'dermapremierskinclinic', False)
('110615337321740', 'dermapremierskinclinic', False)
('101052706033790', 'Covtional', False)
('101052706033790', 'Covtional', False)
('106021138917531', "Amey's Store", False)
('106021138917531', "Amey's Store", False)
('109402158182752', 'Dr Asmaa nabil skin clinic', False)
('109402158182752', 'Dr Asmaa nabil skin clinic', F

('107983035163694', 'Read-Novels-fic', False)
('107983035163694', 'Read-Novels-fic', False)
('116792607687104', 'Fun-Novels4518', False)
('116792607687104', 'Fun-Novels4518', False)
('115413681158429', 'Mo-English-Books', False)
('115413681158429', 'Mo-English-Books', False)
('107983035163694', 'Read-Novels-fic', False)
('107983035163694', 'Read-Novels-fic', False)
('115413681158429', 'Mo-English-Books', False)
('115413681158429', 'Mo-English-Books', False)
('116792607687104', 'Fun-Novels4518', False)
('116792607687104', 'Fun-Novels4518', False)
('108350315163664', 'English-Fun2546', False)
('108350315163664', 'English-Fun2546', False)
('107983035163694', 'Read-Novels-fic', False)
('107983035163694', 'Read-Novels-fic', False)
('116792607687104', 'Fun-Novels4518', False)
('116792607687104', 'Fun-Novels4518', False)
('108350315163664', 'English-Fun2546', False)
('108350315163664', 'English-Fun2546', False)
('108350315163664', 'English-Fun2546', False)
('108350315163664', 'English-Fun2546

('101507555667147', 'Webfic-novels online', False)
('101507555667147', 'Webfic-novels online', False)
('101507555667147', 'Webfic-novels online', False)
('101507555667147', 'Webfic-novels online', False)
('102476542389936', 'Aloha Beauty Clinic-Dr.Abulaban', False)
('102476542389936', 'Aloha Beauty Clinic-Dr.Abulaban', False)
('108464448564605', 'Bookread', False)
('108464448564605', 'Bookread', False)
('1858726777788236', 'Collection', False)
('1858726777788236', 'Collection', False)
('1865825337003993', 'Vendredi', False)
('1865825337003993', 'Vendredi', False)
('1453997771525853', 'Impact Guru', False)
('1453997771525853', 'Impact Guru', False)
('101148695644168', 'Love & Unity TV Network', False)
('101148695644168', 'Love & Unity TV Network', False)
('102945528603307', 'Jeya Journeys', False)
('102945528603307', 'Jeya Journeys', False)
('108464448564605', 'Bookread', False)
('108464448564605', 'Bookread', False)
('108464448564605', 'Bookread', False)
('108464448564605', 'Bookread',

('860795937381795', 'Jesse Make-up Artist', False)
('860795937381795', 'Jesse Make-up Artist', False)
('289248578650135', 'Kiki Riki', False)
('289248578650135', 'Kiki Riki', False)
('110924895729436', 'Cotton Home Hurghada كوتن هوم.', False)
('110924895729436', 'Cotton Home Hurghada كوتن هوم.', False)
('1914601025484059', 'Yassin Diamond', False)
('1914601025484059', 'Yassin Diamond', False)
('1914601025484059', 'Yassin Diamond', False)
('1914601025484059', 'Yassin Diamond', False)
('100652092394539', 'Wave Fit EMS Egypt', False)
('100652092394539', 'Wave Fit EMS Egypt', False)
('100391851831564', 'Car Hospital مستشفى العربيات', False)
('100391851831564', 'Car Hospital مستشفى العربيات', False)
('415697718443863', 'B2B Real Estate', False)
('415697718443863', 'B2B Real Estate', False)
('472421449818130', 'حجابك عنوانك', False)
('472421449818130', 'حجابك عنوانك', False)
('221518554845891', 'الأستاذ / سلطان سعد', False)
('221518554845891', 'الأستاذ / سلطان سعد', False)
('109105098277520'

('259657105893278', 'Shepherd Novel-English', False)
('259657105893278', 'Shepherd Novel-English', False)
('259657105893278', 'Shepherd Novel-English', False)
('106432772145444', 'Pop8494-Novels', False)
('106432772145444', 'Pop8494-Novels', False)
('259657105893278', 'Shepherd Novel-English', False)
('259657105893278', 'Shepherd Novel-English', False)
('101436729289542', 'Etsy.instepwit', False)
('101436729289542', 'Etsy.instepwit', False)
('1905861453048821', 'Cairo_hair_lashes', False)
('1905861453048821', 'Cairo_hair_lashes', False)
('102772075888892', 'glow_with_mona', False)
('102772075888892', 'glow_with_mona', False)
('435135720015294', 'EgySeven Trade-Development', False)
('435135720015294', 'EgySeven Trade-Development', False)
('306665409811223', 'Trinny London', False)
('306665409811223', 'Trinny London', False)
('587496468424460', 'Bescher Beauty', False)
('587496468424460', 'Bescher Beauty', False)
('204965549981252', 'Indagare Natural Beauty', False)
('204965549981252', '

('101148695644168', 'Love & Unity TV Network', False)
('116792607687104', 'Fun-Novels4518', False)
('116792607687104', 'Fun-Novels4518', False)
('116792607687104', 'Fun-Novels4518', False)
('116792607687104', 'Fun-Novels4518', False)
('116792607687104', 'Fun-Novels4518', False)
('116792607687104', 'Fun-Novels4518', False)
('101148695644168', 'Love & Unity TV Network', False)
('101148695644168', 'Love & Unity TV Network', False)
('273710776099378', 'Family Life Church of Abbeville', False)
('273710776099378', 'Family Life Church of Abbeville', False)
('370825939619057', 'Daily Yoga App', False)
('370825939619057', 'Daily Yoga App', False)
('101588395314492', 'Dr. Peter McLewin', False)
('101588395314492', 'Dr. Peter McLewin', False)
('109817891485559', 'Dr.Muscles Gym', False)
('109817891485559', 'Dr.Muscles Gym', False)
('1937962696462346', 'Ingleside Church Norfolk VA', False)
('1937962696462346', 'Ingleside Church Norfolk VA', False)
('2010821535864677', 'Tech Household', False)
('20

('116792607687104', 'Fun-Novels4518', False)
('259657105893278', 'Shepherd Novel-English', False)
('259657105893278', 'Shepherd Novel-English', False)
('259657105893278', 'Shepherd Novel-English', False)
('259657105893278', 'Shepherd Novel-English', False)
('259657105893278', 'Shepherd Novel-English', False)
('259657105893278', 'Shepherd Novel-English', False)
('259657105893278', 'Shepherd Novel-English', False)
('259657105893278', 'Shepherd Novel-English', False)
('259657105893278', 'Shepherd Novel-English', False)
('259657105893278', 'Shepherd Novel-English', False)
('259657105893278', 'Shepherd Novel-English', False)
('259657105893278', 'Shepherd Novel-English', False)
('259657105893278', 'Shepherd Novel-English', False)
('259657105893278', 'Shepherd Novel-English', False)
('259657105893278', 'Shepherd Novel-English', False)
('259657105893278', 'Shepherd Novel-English', False)
('259657105893278', 'Shepherd Novel-English', False)
('259657105893278', 'Shepherd Novel-English', False)
(

('44389181906', 'BehindTheChair.com', False)
('44389181906', 'BehindTheChair.com', False)
('126983971342683', 'Vivoo', False)
('126983971342683', 'Vivoo', False)
('104568724641042', 'CSM-Canadian Spectrum Media INC.', False)
('104568724641042', 'CSM-Canadian Spectrum Media INC.', False)
('111411471422303', 'Dr.Asmaa Ragheb', False)
('111411471422303', 'Dr.Asmaa Ragheb', False)
('100110425921069', 'Mattch-better.US', False)
('100110425921069', 'Mattch-better.US', False)
('100110425921069', 'Mattch-better.US', False)
('100110425921069', 'Mattch-better.US', False)
('100110425921069', 'Mattch-better.US', False)
('100110425921069', 'Mattch-better.US', False)
('294854275656518', 'الملكه للمفروشات', False)
('294854275656518', 'الملكه للمفروشات', False)
('1677619875609788', 'ZEE Designs', False)
('1677619875609788', 'ZEE Designs', False)
('102637462505119', 'Capriceapp-X', False)
('102637462505119', 'Capriceapp-X', False)
('2323508577890827', 'Sleep Booster - Sleep Better', False)
('2323508577

In [30]:
import sqlite3
sqliteConnection = sqlite3.connect('FaceBoookADS.db')
cursor = sqliteConnection.cursor()
for record in records:
    try:
        sqlite_select_query = f"""INSERT INTO facebook_pages values(?,?,?) """
        cursor.execute(sqlite_select_query,(record[0],record[1],False))
        sqliteConnection.commit()
    except:
        pass
cursor.close()

In [8]:
records

['798240993621809',
 '1149310848422401',
 '111235721585655',
 '111235721585655',
 '101424012697046',
 '101424012697046',
 '101424012697046',
 '110615337321740',
 '101052706033790',
 '106021138917531',
 '109402158182752',
 '101240422184383',
 '105597255558211',
 '112689744739890',
 '112689744739890',
 '102768482503678',
 '111266694917097',
 '107782564896443',
 '106459458735935',
 '100263112783558',
 '100263112783558',
 '100263112783558',
 '100263112783558',
 '100263112783558',
 '100263112783558',
 '100263112783558',
 '100263112783558',
 '100263112783558',
 '100263112783558',
 '100263112783558',
 '100263112783558',
 '107401478526785',
 '101464925740704',
 '101464925740704',
 '101464925740704',
 '101464925740704',
 '101464925740704',
 '105880582872515',
 '1453997771525853',
 '107458688711965',
 '102641208320329',
 '102641208320329',
 '102641208320329',
 '105150181176221',
 '105150181176221',
 '1453997771525853',
 '1453997771525853',
 '1453997771525853',
 '1453997771525853',
 '145399777152

In [ ]:
import sqlite3
sqliteConnection = sqlite3.connect('FaceBoookADS.db')
cursor = sqliteConnection.cursor()
col = 'AD_ID'
sqlite_select_query = f"""SELECT facebook_pages.static FROM ads where
(
(null is null) )
"""
sqlite_select_query
cursor.execute(sqlite_select_query)
records = cursor.fetchall()
print(len(records))
cursor.close()

In [ ]:
Blog.objects.filter(entry__headline__contains='Lennon').filter(entry__pub_date__year=2008)


In [ ]:
facebook_pages.objects.select_related('reporter') #


In [53]:
Ads.objects.filter(favourite=True).order_by('-ads_count')

FieldError: Cannot resolve keyword 'favourite' into field. Choices are: ad_id, ad_occurance, ads_count, content, cumulative_ads_count, date, days, facebook_id, footer_action, footer_text, hits, insta_followers, instgram_id, links, page_likes, page_name, profile_pic, search_term, started_date, static_id, videos

In [34]:
ADS = Ads.objects.all().order_by('-ads_count')
# days = [len(set(rec.days.split(','))) for rec in ADS]
# cond_list = [True if i > 2 else False for i in days]
# ADS  = list(compress(ADS,cond_list))

In [44]:
advertisers_side = [link.page_name for link in ADS] #ADS.values_list('links') ADS.values_list('page_name')
advertisers_side = [i for i in advertisers_side]
advertisers_side = dict(sorted(Counter(advertisers_side).items(), key=lambda item: item[1],reverse=True))

# advertisers_side = dict(itertools.islice(sorted_dict.items()))

In [45]:
advertisers_side

{'Fun-Novels4518': 48,
 'Shepherd Novel-English': 47,
 'Read-Novels-fic': 37,
 'Mo-English-Books': 33,
 'ILoveNovel': 33,
 'Dr. Squatch': 30,
 'Novel2': 24,
 'Pop8891-Read&Write Novels': 23,
 'Bookread': 23,
 'Pop8494-Novels': 22,
 'Romance novel': 21,
 'Dr Naomi Skin': 21,
 'English-Fun2546': 20,
 'Joynovel Stories': 18,
 'ال_عمر': 17,
 'Perfume.com': 17,
 'Eternal Clear': 16,
 'The world of reader': 15,
 'Lumae Skin': 13,
 'Pop8268-Stories': 12,
 'F-111': 12,
 'Furnish Up Rollic': 12,
 'Webfic': 11,
 'Impact Guru': 10,
 'KeenNovels': 10,
 'تاريخ و كورافيا مع شادي حبشي': 10,
 'Novel-3721': 10,
 'شركة ال_عمرللتوريدات': 9,
 'ابطال اوروبا بالعربية': 9,
 'esmi Skin Minerals': 9,
 'Shopping Drama': 8,
 'The world of read': 8,
 'Mazaya Stores': 8,
 'Keenovel': 8,
 'Interesting novel': 7,
 'Mindvalley': 7,
 'Cordus & Sacrus: The Solution to Back Pain': 7,
 'The Habit': 7,
 'Primalderm 2.0': 7,
 'Forsetk - فرصتك': 6,
 'Noveltells': 6,
 'My Wallet - محفظتي': 6,
 'Qure Skincare': 6,
 'HeySilkyS

In [8]:
ADS = Ads.objects.all()

In [15]:

advertisers

{'Fun-Novels4518': 48,
 'Shepherd Novel-English': 47,
 'Read-Novels-fic': 37,
 'Mo-English-Books': 33,
 'ILoveNovel': 33}

In [92]:
ADS = Ads.objects.all().filter(ad_id = '1024108578278285')

In [98]:
ADS.values_list('links')[0][0].split()

['https://ariadneclothing.com/?fbclid=IwAR1oMp1gzOqdL5YusyIibjk1yThzx7JYo0Jfoi031e7YbaKXrW626ZyscLY',
 'https://ariadneclothing.com/?fbclid=IwAR252eY-x1oqhCaqNAhd6AMIqka78HD-z4I_xIkjYWBkEgrEaz_sF-e1D-I']

In [106]:
ADS = Ads.objects.all()
vids = [len(set(rec.videos.split())) for rec in ADS]
cond_list = [True if i > 1 else False for i in vids]
ADS1 = list(compress(ADS,cond_list))

In [108]:
for i in ADS1:
    print(i.page_name)

Premedion Health & Wellbeing
اطلب لنا
SAMIR Stores
ال_عمر
ال_عمر
ال_عمر
شركة ال_عمرللتوريدات
شركة ال_عمرللتوريدات
شركة ال_عمرللتوريدات
ال_عمر
شركة ال_عمرللتوريدات
شركة ال_عمرللتوريدات
ال_عمر
ال_عمر
ال_عمر
ال_عمر
ال_عمر
ال_عمر
Much Better Adventures
Sebastian Ucha


In [64]:
from django.db.models.functions import Greatest

Ads.objects.annotate(
    latest_activity_at=Greatest('page_likes', 'insta_followers')
).filter(latest_activity_at__gte = 20000).order_by('latest_activity_at').values_list('page_likes', 'insta_followers')

<QuerySet [(20077, 0), (20141, 4402), (0, 20575), (0, 20632), (20854, 145), (20854, 145), (20854, 145), (20854, 145), (20854, 145), (20854, 145), (20854, 145), (0, 21070), (21234, 3779), (0, 21513), (2470, 21649), (18846, 21693), (0, 21879), (0, 21936), (21942, 3746), (22008, 1605), '...(remaining elements truncated)...']>

In [30]:
ADS = Ads.objects.all()
days = []
for rec in ADS:
    days.append(len(set(rec.days.split(','))))
    
days_running = 4
cond_list = [True if i > days_running else False for i in days]

# ADS1 = compress(ADS,cond_list)

In [7]:
ADS[0].facebook_ida

'@esmiskinminerals'

In [89]:
ads_count = 10

In [84]:
()

21

In [18]:
ad_id = started_date = profile_pic = links = videos = content = footer_text = footer_action = page_name = ad_occurance = facebook_id = page_likes = instgram_id = insta_followers = static_id = ads_count = cumulative_ads_count = days = date = hits = search_term = 'null'

In [80]:
for ad in Ads.objects.raw('''select * from ads limit 5''').defer(None):
    print(ad)
# for ad in ADS:
#     print(ad.ad_id)

AttributeError: 'RawQuerySet' object has no attribute 'defer'

In [32]:
if "varr4" 
    print(1)

In [44]:
import sqlite3
sqliteConnection = sqlite3.connect('FaceBoookADS.db')
cursor = sqliteConnection.cursor()
col = 'AD_ID'
# sqlite_select_query = f"""SELECT * FROM ads where
# (
# (null is null) )
# """
sqlite_select_query
cursor.execute(sqlite_select_query)
records = cursor.fetchall()
print(len(records))
cursor.close()

0


In [42]:
if 'Facebook_ID' in (globals() or locals()):
    sqlite_select_query += f'and ({Facebook_ID} is null OR Facebook_ID = {Facebook_ID})'

In [43]:
sqlite_select_query

'SELECT * FROM ads where\n(\n(null is null) )\nand (20 is null OR Facebook_ID = 20)'

In [60]:
from django.db import connection

def my_custom_sql(self):
    with connection.cursor() as cursor:
        cursor.execute("SELECT * FROM ads")
        row = cursor.fetchone()

    return row

In [61]:
with connection.cursor() as cursor:
    cursor.execute("SELECT * FROM ads")
    row = cursor.fetchone()
row

SynchronousOnlyOperation: You cannot call this from an async context - use a thread or sync_to_async.

In [8]:
import sqlite3
sqliteConnection = sqlite3.connect('FaceBoookADS.db')
cursor = sqliteConnection.cursor()
sqlite_select_query = f"""SELECT static_ID, instgram_ID,Facebook_ID FROM ads where instgram_ID = 'NO insta ID found' and Facebook_ID = 'NO Facebook ID found' """
cursor.execute(sqlite_select_query)
records = cursor.fetchall()
page_IDS = [i[0] for i in list(set(records))]
cursor.close()
for page_ID in page_IDS:
    open_page(page_ID)
    Ads_count = get_ads_number()
    conn = sqlite3.connect('FaceBoookADS.db')
    c = conn.cursor()
    c.execute('''UPDATE ads SET 
    Ads_count =?,
    cumulative_ads_count = cumulative_ads_count + ?
    where static_id = ?'''
    ,(
    Ads_count ,
    Ads_count,
    page_ID))
    conn.commit()
    conn.close()

## video Time

In [7]:
import cv2
import datetime
 
# create video capture object
data = cv2.VideoCapture('https://video.fcai19-1.fna.fbcdn.net/v/t42.9040-2/10000000_1015749882158468_3760737243426193408_n.mp4?_nc_cat=102&ccb=1-7&_nc_sid=cf96c8&_nc_eui2=AeE_30sbGlJJBlNjbXjLe2CYHdEfWt8VkaYd0R9a3xWRpvNv693O_Z-sOwfGvX2Mq433H9GBnOQS-6NTco1svKAn&_nc_ohc=JKfR2vhz59oAX-a9bbU&_nc_ht=video.fcai19-1.fna&oh=00_AT-Xr0fUWmJerCUPYSrVGbRi-93squGhxupz7JpMDMv_Ig&oe=6324FF5F')
 
# count the number of frames
frames = data.get(cv2.CAP_PROP_FRAME_COUNT)
fps = data.get(cv2.CAP_PROP_FPS)
 
# calculate duration of the video
seconds = round(frames / fps)
video_time = datetime.timedelta(seconds=seconds)
print(f"duration in seconds: {seconds}")
print(f"video time: {video_time}")

duration in seconds: 534
video time: 0:08:54


In [46]:
def return2():
    return 1 ,2

In [49]:
a,b = return2()

In [65]:
sqliteConnection = sqlite3.connect('FaceBoookADS.db')
cursor = sqliteConnection.cursor()
sqlite_select_query = f"""SELECT search_term,country FROM search_terms where active = True and search_type = 'keyword'"""
cursor.execute(sqlite_select_query)
terms_DB = cursor.fetchall()
terms = [i[0] for i in terms_DB]
terms_countries =  [i[1] for i in terms_DB] 
# terms_countries = [i[1] for i in terms_DB]
sqliteConnection.close()

In [73]:
for term,country in zip(terms,terms_countries):
    print(term,country)
    

عطور ALL
asd ALL
ABC ALL
asd EG


In [68]:
terms_countries

['ALL', 'ALL', 'ALL', 'EG']